# Structured ouput

Les LLM ne se limitent pas à produire du texte libre : ils peuvent aussi générer des données structurées, comme du JSON.
Cela permet de standardiser les réponses, faciliter leur exploitation par du code, ou les intégrer dans des systèmes existants.
On peut ainsi extraire des informations, classer des contenus, ou même générer des objets conformes à un schéma métier.
En exploitant cette capacité, on transforme un modèle de langage en outil d'automatisation robuste et interopérable.

### Dépendances et clé d'API

In [ ]:
#Décommenter sur Google Colab
#%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')


#Décommenter en local
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")

## Génération de données synthétiques

Un premier cas d'usage est de faire générer des données synthétiques structurées au modèle, par exemple pour générer un liste d'utilisateur, de produit, etc.

In [1]:
from pydantic import BaseModel, Field
from typing import List, Literal, Optional

class NPCSimple(BaseModel):
    name: str = Field(..., description="Nom du personnage")
    race: str = Field(..., description="Race du personnage")
    skills: List[str]

class NPCSimpleList(BaseModel):
    npcs: List[NPCSimple] = Field(..., description="Liste de personnages non-joueurs (PNJ)")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai").with_structured_output(NPCSimpleList)

response = llm.invoke("Génère une liste de 3 personnages non-joueurs (PNJ) pour un jeu de rôle médiéval fantastique.") 

response


NPCSimpleList(npcs=[NPCSimple(name='Elda la Sage', race='Elfe', skills=['Connaissance des herbes', 'Magie élémentaire', 'Diplomatie']), NPCSimple(name='Brom le Brave', race='Nain', skills=['Combat à la hache', 'Forge', 'Résistance à la douleur']), NPCSimple(name='Lyra la Mystique', race='Humaine', skills=['Charme', 'Illusions', "Détournement d'attention"])])

Beaucoup des subtilités permises par la définition de schéma `pydantic` (`zod`en js/ts) peuvent être exploités pour guider au mieux la génération.

In [2]:


# Pydantic
class NPC(BaseModel):
    name: str = Field(..., description="Nom du personnage")
    race: Literal["humain", "elfe", "nain", "orc", "gobelin", "tieffelin", "dragonide"] = Field(
        ..., description="Race du personnage"
    )
    role: Literal["marchand", "guerrier", "mage", "voleur", "barde", "alchimiste", "prêtre"] = Field(
        ..., description="Rôle ou classe du personnage"
    )
    alignment: Literal["bon", "neutre", "mauvais"] = Field(..., description="Alignement moral")
    level: int = Field(..., ge=1, le=20, description="Niveau de puissance du personnage")
    description: Optional[str] = Field(None, description="Courte description physique et comportementale")
    skills: List[str] = Field(default_factory=list, description="Compétences principales du personnage")
    inventory: List[str] = Field(default_factory=list, description="Objets portés par le personnage")
    backstory: Optional[str] = Field(None, description="Origine ou histoire personnelle du personnage")

    class Config:
        schema_extra = {
            "example": {
                "name": "Thalior Ombrecendre",
                "race": "elfe",
                "role": "assassin",
                "alignment": "neutre",
                "level": 7,
                "description": "Un elfe aux yeux sombres, silencieux et agile.",
                "skills": ["furtivité", "armes légères", "persuasion"],
                "inventory": ["dague empoisonnée", "cape d'ombre", "bourse vide"],
                "backstory": "Ancien espion au service du royaume déchu d’Ardor. Il vend aujourd’hui ses services au plus offrant."
            }
        }

class NPCList(BaseModel):
    npcs: List[NPC] = Field(..., description="Liste de personnages non-joueurs (PNJ)")

llm = init_chat_model("gpt-4o-mini", model_provider="openai").with_structured_output(NPCList)

response = llm.invoke("Génère une liste de 2 personnages non-joueurs (PNJ) pour un jeu de rôle médiéval fantastique.") 

list_npc = NPCList.model_validate(response)

print(list_npc)
print(f"NPC: {list_npc.npcs[0].name,} - {list_npc.npcs[1].name}")



/home/bredda/workspace/formation_ia/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
/home/bredda/workspace/formation_ia/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
/home/bredda/workspace/formation_ia/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


npcs=[NPC(name='Aldrin le Sage', race='humain', role='mage', alignment='neutre', level=8, description='Un vieil homme avec une longue barbe blanche et des yeux perçants. Il porte une robe bleu nuit ornée de symboles mystiques.', skills=['magie élémentaire', 'alchemy', 'lore'], inventory=['grimoire ancien', 'poudre magique', 'cristal de mana'], backstory="Aldrin a consacré sa vie à l'étude des arcanes et a voyagé à travers de nombreux royaumes à la recherche de sagesse."), NPC(name='Gruk le Fort', race='orc', role='guerrier', alignment='bon', level=6, description='Un orc musclé avec une peau verte et des cicatrices, il porte une armure faite de cuir renforcé et brandit une hache imposante.', skills=['combat au corps à corps', 'résilience', 'tactique de bataille'], inventory=['hache massive', 'armure en cuir', 'potion de soin'], backstory="Gruk a été rejeté par son clan pour avoir défendu les innocents lors d'un raid, il se bat désormais pour protéger les faibles.")]
NPC: ('Aldrin le Sag

## Extraction de données en input

In [4]:
inputs = [
"""
Objet : Urgence - Impossible de se connecter à l'intranet

Bonjour,

Depuis ce matin, aucun collaborateur ne parvient à accéder à l’intranet de l’entreprise. Un message d’erreur « 502 Bad Gateway » s’affiche systématiquement.

Merci de résoudre cela au plus vite, c’est bloquant pour toute l’équipe RH.

Cordialement,  
Caroline Dumas
""",
"""
Sujet : Demande d’accès à un nouveau logiciel

Bonjour,  
Je souhaiterais pouvoir utiliser l’outil Notion pour la gestion de mes projets.  
Pouvez-vous procéder à l’installation ou me dire quelle est la démarche à suivre ?  

Merci d’avance,  
Mohamed
"""]

from pydantic import BaseModel, Field
from typing import Literal


class SupportTicket(BaseModel):
    type: Literal["incident", "demande", "question", "autre"] = Field(..., description="Type de ticket")
    priorité: Literal["basse", "moyenne", "haute", "critique"] = Field(..., description="Niveau de priorité")
    sujet: str = Field(..., description="Résumé court du problème ou de la demande")
    service_concerné: Literal["IT", "RH", "Finances", "Achats", "Inconnu"] = Field(..., description="Domaine concerné")
    requiert_action_humaine: bool = Field(..., description="Nécessite une action manuelle immédiate")

llm = init_chat_model("gpt-4o-mini", model_provider="openai").with_structured_output(SupportTicket)

for input_text in inputs:
    response = llm.invoke(input_text)
    ticket = SupportTicket.model_validate(response)
    print(f"Type: {ticket.type}, Priorité: {ticket.priorité}, Sujet: {ticket.sujet}, Service: {ticket.service_concerné}, Action requise: {ticket.requiert_action_humaine}")

Type: incident, Priorité: critique, Sujet: Accès impossible à l’intranet - Erreur 502 Bad Gateway, Service: IT, Action requise: True
Type: demande, Priorité: moyenne, Sujet: Demande d’accès à Notion pour la gestion de projets, Service: IT, Action requise: True


## Validation et normalisation de données naturelles



In [5]:
inputs = [
    "Je veux réserver une salle de réunion pour demain à 8h30 pour 2h.",
    "demin midi por max 1 heure",
]

from pydantic import BaseModel, Field

class MeetingRoomBooking(BaseModel):
    date: str = Field(..., description="Date de la réservation au format YYYY-MM-DD")
    start_time: str = Field(..., description="Heure de début au format HH:MM")
    duration_hours: int = Field(..., ge=1, le=8, description="Durée de la réservation en heures")

llm = init_chat_model("gpt-4o-mini", model_provider="openai").with_structured_output(MeetingRoomBooking)

for input_text in inputs:
    response = llm.invoke(input_text)
    ticket = MeetingRoomBooking.model_validate(response)
    print(f"Date: {ticket.date}, Heure de début: {ticket.start_time}, Durée: {ticket.duration_hours} heures")

Date: 2023-10-06, Heure de début: 08:30, Durée: 2 heures
Date: 2023-10-06, Heure de début: 12:00, Durée: 1 heures
